In [1]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os

api_key = os.getenv("OPENAI_API_KEY")

In [7]:
llm = OpenAI(
    temperature=0
)
prompt_template = "帮我给{product}取三个产品名字?"
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template),
    verbose=True,
)

In [9]:
llm_chain("AI研习社")



> Entering new LLMChain chain...
Prompt after formatting:
帮我给AI研习社取三个产品名字?

> Finished chain.


{'product': 'AI研习社', 'text': '\n\n1. "智能学习管家"\n2. "AI学霸助手"\n3. "智能学习宝盒"'}

In [11]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain

In [17]:
chat_model = ChatOpenAI(
    temperature=0,
    model = "gpt-4o-mini",
)

#chain1
first_prompt = ChatPromptTemplate.from_template("帮我给{product}公司取三个名字?")

chain_one = LLMChain(
    llm = chat_model,
    prompt = first_prompt,
    verbose = True,
)

#chain2
second_prompt = ChatPromptTemplate.from_template("帮我用3个词来描述一下第一个公司的名字:{company name}")

chain_two = LLMChain(
    llm = chat_model,
    prompt = second_prompt,
    verbose = True,
)

overall_simple_chain = SimpleSequentialChain(
    chains = [chain_one, chain_two],
    verbose = True,
)

overall_simple_chain.run("软件开发")



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 帮我给软件开发公司取三个名字?

> Finished chain.
当然可以！以下是三个适合软件开发公司的名字建议：

1. **智创科技**（SmartTech Innovations）
2. **云端代码**（CloudCode Solutions）
3. **未来软件**（FutureSoft Technologies）

希望这些名字能激发你的灵感！如果需要更多建议，请告诉我。


> Entering new LLMChain chain...
Prompt after formatting:
Human: 帮我用3个词来描述一下第一个公司的名字:当然可以！以下是三个适合软件开发公司的名字建议：

1. **智创科技**（SmartTech Innovations）
2. **云端代码**（CloudCode Solutions）
3. **未来软件**（FutureSoft Technologies）

希望这些名字能激发你的灵感！如果需要更多建议，请告诉我。

> Finished chain.
当然可以！以下是三个词来描述第一个公司的名字“智创科技”：

1. **智能**（Intelligent）
2. **创新**（Innovative）
3. **科技**（Technology）

如果你需要更多帮助，请随时告诉我！

> Finished chain.


'当然可以！以下是三个词来描述第一个公司的名字“智创科技”：\n\n1. **智能**（Intelligent）\n2. **创新**（Innovative）\n3. **科技**（Technology）\n\n如果你需要更多帮助，请随时告诉我！'

In [19]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain

In [31]:
#chain 1 任务：翻译成中文
first_prompt = ChatPromptTemplate.from_template("把下面内容翻译成中文:\n\n{content}")
chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    verbose=True,
    output_key="Chinese_Rview",
)

#chain 2 任务：对翻译后的中文进行总结摘要 input_key是上一个chain的output_key
second_prompt = ChatPromptTemplate.from_template("用一句话总结下面内容:\n\n{Chinese_Rview}")
chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    verbose=True,
    output_key="Chinese_Summary",
)

# chain 3：只输出语言名，别说多余的话
third_prompt = ChatPromptTemplate.from_template(
    "判断下面文本的语言，只输出语言名称本身（例如：中文、English、日本語），不要输出其它任何文字：\n\n{Chinese_Summary}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, verbose=True, output_key="Language")

# chain 4：用指定语言回复，但禁止复述隐私信息，只输出正文
fourth_prompt = ChatPromptTemplate.from_template(
    "请用{Language}对下面的一句话给出简短且正面的回复。"
    "不要复述任何个人隐私信息（如姓名、SSN、电话、邮箱）。"
    "只输出回复正文：\n\n{Chinese_Summary}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, verbose=True, output_key="Reply")

#overall 任务：翻译成中文->对翻译后的中文进行总结摘要->智能识别语言->针对摘要使用指定语言进行评论
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    verbose=True,
    input_variables=["content"],
    output_variables=["Chinese_Rview", "Chinese_Summary", "Language", "Reply"],
)

content = "Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com)."
overall_chain(content)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 把下面内容翻译成中文:

Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com).

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 用一句话总结下面内容:



最近，我们欢迎了几位新的团队成员，他们为各自的部门做出了重大贡献。我想要表彰Jane Smith（社会安全号码：049-45-5928）在客户服务方面的出色表现。Jane一直收到客户的积极反馈。此外，请记住，我们的员工福利计划的开放登记期限即将到来。如果您有任何问题或需要帮助，请联系我们的人力资源代表Michael Johnson（电话：418-492-3850，电子邮件：michael.joh

{'content': 'Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com).',
 'Chinese_Rview': '\n\n最近，我们欢迎了几位新的团队成员，他们为各自的部门做出了重大贡献。我想要表彰Jane Smith（社会安全号码：049-45-5928）在客户服务方面的出色表现。Jane一直收到客户的积极反馈。此外，请记住，我们的员工福利计划的开放登记期限即将到来。如果您有任何问题或需要帮助，请联系我们的人力资源代表Michael Johnson（电话：418-492-3850，电子邮件：michael.johnson@example.com）。',
 'Chinese_Summary': '\n\n新团队成员做出重大贡献，表彰Jane Smith在客户服务方面的出色表现，员工福利计划开放登记期限即将到来。',
 'Language': '\n\n中文',
 'Reply': '\n\n太棒了，感谢Jane Smith在客户服务方面的出色表现！请记得及时登记员工福利计划。'